### XGBoost for Time Series Forecasting

Using XGBoost (one of the variant of Boosting) for forecasting stock price. [XGBoost documentation](https://xgboost.readthedocs.io/en/latest/)
- XGBoost is an ensemble of decision trees where new trees fix errors of the trees that are already part of the model. It is therefore, trees are added until no further improvements can be made to the model.
- In order to use XGBoost for time series, we need to evaluate the model via `walk-forward validation` instead of `k-fold cross validation` because k-fold sometimes would have biased results.

#### Importing Extensions and Libraries

In [1]:
%load_ext autoreload
%load_ext watermark
%autoreload 2
%load_ext lab_black

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time

plt.style.use(style="seaborn")
%matplotlib inline

In [3]:
%watermark -iv -v

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.22.0

matplotlib: 3.4.1
numpy     : 1.20.2
pandas    : 1.2.3

